In [14]:
!pip install openpyxl
from google.colab import files

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pandas as pd
import random
from google.colab import data_table
data_table.enable_dataframe_formatter()
df = pd.read_excel("/content/drive/MyDrive/Lottery/data.xlsx")
df.head(2)

,Identified By,Identified by EmployeeID,Event,Business Area,Division,District,Facilty/Pipeline,Location,Title,Event Date,...,Employee ID (Person Involved),Titles of Person Involved,Relation of Person involved,Engineering Contractors,Ops Contractors,Completed Date,DOT Recordable,OSHA Recordable,Root Cause,Cause Category
0,Andrea J James,785597,EVT132867,Pipeline,Gulf Coast PL,YORKTOWN TERMINAL,YORKTOWN TERMINAL,YORKTOWN TERMINAL / ADMIN BLDG,Liquid on the Floor,2022-01-28 08:52:59.999,...,785597.0,NaN,Employee,0,0,2022-01-28 09:03:52.999,NaN,NaN,NaN,NaN
1,Daniel V Juarez,789605,EVT132117,Pipeline,Southwestern PL,PERMIAN PIPELINE PECOS,NaN,PPP,COV -Deer strike,2022-01-04 18:00:00.000,...,6778.0,PIPELINE OPERATOR,Employee,0,0,2022-01-11 17:06:55.999,NaN,NaN,NaN,NaN


In [5]:
df = df[["Event", "Control", "Division", "District", "Identified By", "Identified by EmployeeID"]]
df = df.rename({"Identified By" : "name", "Identified by EmployeeID" : "id"}, axis = 1)
# Next, fill every id with leading zeros for the id later read properly and lessen the workload for Payroll department
# Notice that employees id at index 2 is not 6 digit
df.head(5)

,Event,Control,Division,District,name,id
0,EVT132867,No,Gulf Coast PL,YORKTOWN TERMINAL,Andrea J James,785597
1,EVT132117,No,Southwestern PL,PERMIAN PIPELINE PECOS,Daniel V Juarez,789605
2,EVT134668,No,Southwestern PL,PERMIAN PIPELINE PECOS,David Purdy,6851
3,EVT133282,No,Mid Continent PL,CHEROKEE PL,Dessie L Elledge,789332
4,EVT132072,No,Mid Continent PL,CONWAY PL,Jay D Burgin,789559


In [3]:
# Change the column "id" in df to string type
df.id = df.id.astype(str)
for i, row in df.iterrows():
    # str(row.id).zfill(6)
    df.at[i,'id'] = str(row.id).zfill(6)
df.head(5)

,Event,Control,Division,District,name,id
0,EVT132867,No,Gulf Coast PL,YORKTOWN TERMINAL,Andrea J James,785597
1,EVT132117,No,Southwestern PL,PERMIAN PIPELINE PECOS,Daniel V Juarez,789605
2,EVT134668,No,Southwestern PL,PERMIAN PIPELINE PECOS,David Purdy,006851
3,EVT133282,No,Mid Continent PL,CHEROKEE PL,Dessie L Elledge,789332
4,EVT132072,No,Mid Continent PL,CONWAY PL,Jay D Burgin,789559


In [6]:
totalEmployees = len(set(df["id"]))
totalEntries = len(df["Event"])
print(totalEmployees)
print(totalEntries)

550
2826


In [8]:
visited = set()
division = []
district = []
control = []
name = []
events = [[]]
budget = 45500
total = 0
chosen = {}
index = random.randint(0, totalEntries)
while True:
    while index in visited:
        index = random.randint(0, totalEntries)
    visited.add(index)
    if df["id"][index] not in chosen.keys():
        value = 500 if df["Control"][index].lower() == "no" else 1000
        # In case if the value 1000 and total is 44500 and the budget is 45000, we can obviously can fit one more 500 entries
        # As a result, skip to go to the next entry
        if total + value > budget:
            if budget - total == 500:
              continue
            else:
              break
        chosen[df["id"][index]] = value
        events.append([df["Event"][index]])
        division.append(df["Division"][index])
        district.append(df["District"][index])
        name.append(df["name"][index])
        control.append(df["Control"][index])
        total += value
    elif chosen[df["id"][index]] != 0 and chosen[df["id"][index]] < 1000:
        value = 1000 - chosen[df["id"][index]]
        if total + value > budget:
            break
        chosen[df["id"][index]] = 1000
        events[list(chosen.keys()).index(df["id"][index])+1].append(df["Event"][index])
        total += value
events.pop(0)

[]

In [32]:
print(total)
# Convert the dictionary to pandas data frame
final_df = pd.DataFrame(chosen.items(), columns=['Identified by EmployeeID', 'Wining Amount'])
final_df = final_df.assign(Division = division, District = district, Name = name, Control = control, Event = events)
final_df = final_df.rename({"Name" : "Identified By"}, axis = 1)
final_df = final_df[["Event", "Control", "Division", "District", "Identified By", "Identified by EmployeeID", "Wining Amount"]]


# Change the column "Identified by EmployeeID" in final_df to string type
final_df["Identified by EmployeeID"] = final_df["Identified by EmployeeID"].astype(str)
for i, row in final_df.iterrows():
#     # str(row.id).zfill(6)
    final_df.at[i,'Identified by EmployeeID'] = str(row['Identified by EmployeeID']).zfill(6)


final_df

45500


,Event,Control,Division,District,Identified By,Identified by EmployeeID,Wining Amount
0,[EVT132444],No,Southwestern PL,PERMIAN NORTH PL,Clayton M Hook,785051,500
1,[EVT134327],Yes,Southwestern PL,PERMIAN EAST PL,Joel G Phelps,050449,1000
2,[EVT135350],No,Southwestern PL,PERMIAN SOUTH PL,Zachary I Guerrero,790620,500
3,[EVT135584],Yes,Mid Continent PL,CHEROKEE PL,Danny M Alexander,780720,1000
4,[EVT135248],Yes,Mid Continent PL,COLORADO CITY PL,Don L Timora,781798,1000
...,...,...,...,...,...,...,...
65,[EVT135997],No,Mid Continent PL,CUSHING TERMINAL,Kevin H Cearley,780584,500
66,[EVT133754],Yes,Southwestern PL,PERMIAN NORTH PL,Johnny W Diaz,790523,1000
67,[EVT135991],No,Mid Continent PL,CHEROKEE PL,Robert F Lama,781794,500
68,[EVT132126],No,Southwestern PL,PERMIAN SOUTH PL,Sesario S Quintana,789544,500


In [33]:
final_df.to_excel("lottery.xlsx")
# final_df.to_excel(r'C:\export_dataframe.xlsx', index = False, header=True)

files.download('lottery.xlsx') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>